In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecell_tracker import segment
from livecell_tracker import core
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic
# import detectron2
# from detectron2.utils.logger import setup_logger

# setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import cv2

# import some common detectron2 utilities
# from detectron2 import model_zoo
# from detectron2.engine import DefaultPredictor
# from detectron2.config import get_cfg
# from detectron2.utils.visualizer import Visualizer
# from detectron2.data import MetadataCatalog, DatasetCatalog
# from livecell_tracker.segment.detectron_utils import gen_cfg

# from livecell_tracker.segment.detectron_utils import (
#     segment_detectron_wrapper,
#     segment_images_by_detectron,
#     convert_detectron_instance_pred_masks_to_binary_masks,
#     convert_detectron_instances_to_label_masks,
# )
# from livecell_tracker.segment.detectron_utils import (
#     convert_detectron_instance_pred_masks_to_binary_masks,
#     convert_detectron_instances_to_label_masks,
#     segment_images_by_detectron,
#     segment_single_img_by_detectron_wrapper,
# )


## Loading single cells from existing mask files

```LiveCellImageDataset, SingleCellImageDataset``` from livecell_tracker.core.datasets allow users to load images with ease, without reading directly into memories.  
In `mask_dataset_path` please make sure that the sorted (alphabetically) file names correspond to the order of times.
 Note that the sorted mechanism provided is simply sort the url (file name) list according to string value. Please note that without proper left trailing zeroes, the order of final times may be incorrect. E.g. string  `T10` (10th file) is less than string `T2`. If you have your customized file patterns, please provide `LiveCellImageDataset` with a `time2url` dictionary to provide necessary time information mapped to file locations for reading time-lapsed data.
`SingleCellImageDataset` takes a single image from the memory and makes it a single time point dataset, which can be handy when you would like to process imaging datasets. 

In [ ]:
dataset_dir_path = Path(
    "../datasets/test_data_STAV-A549/DIC_data"
)

mask_dataset_path = Path("../datasets/test_data_STAV-A549/mask_data")

In [ ]:
mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
mask_dataset.time2url

In [ ]:
dic_dataset = LiveCellImageDataset(dataset_dir_path, ext="tif")

Check if the `time2url` mapping is correct

In [ ]:
dic_dataset.time2url

### Convert label masks to single objects

In [ ]:
from skimage.measure import regionprops
from livecell_tracker.segment.utils import prep_scs_from_mask_dataset
single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

In [ ]:
for sc in single_cells:
    assert sc.mask_dataset

In [ ]:
# for testing
# single_cells = single_cells[:10]

In [ ]:
len(single_cells)

In [ ]:
single_cells_by_time = {}
for cell in single_cells:
    if cell.timeframe not in single_cells_by_time:
        single_cells_by_time[cell.timeframe] = []
    single_cells_by_time[cell.timeframe].append(cell)

In [ ]:
for time in single_cells_by_time:
    print(time, len(single_cells_by_time[time]))

### Visualize one single cell

In [ ]:
sc = single_cells[0]

fig, axes = plt.subplots(1, 4, figsize=(10, 5))
sc.show(ax=axes[0])
sc.show_mask(ax=axes[1])
sc.show_contour_img(ax=axes[2])
sc.show_contour_mask(ax=axes[3])

In [ ]:
sc.show_panel(figsize=(15, 5))

In [ ]:
sc1 = single_cells[1]
sc2 = single_cells[2]

In [ ]:
from livecell_tracker.trajectory.feature_extractors import compute_skimage_regionprops, compute_haralick_features

skimage_features = compute_skimage_regionprops(sc1)
sc1.add_feature("skimage", skimage_features)

In [ ]:
# haralick_features = compute_haralick_features(sc1)
# sc1.add_feature("haralick", haralick_features)

In [ ]:
sc1.get_feature_pd_series()

Calculate overlap between two single cells

In [ ]:
sc1.compute_iou(sc2), sc1.compute_overlap_percent(sc2)

## Tracking based on single cells

In [ ]:
from typing import List
from livecell_tracker.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)


traj_collection = track_SORT_bbox_from_scs(single_cells, dic_dataset, mask_dataset=mask_dataset, max_age=1, min_hits=1)

generate movies

In [ ]:
# from livecell_tracker.track.movie import generate_single_trajectory_movie

# for track_id, traj in traj_collection:
#     generate_single_trajectory_movie(traj, save_path=f"./notebook_results/general_tutorial/track_movies/track_{track_id}.gif")

In [ ]:
traj_collection.histogram_traj_length()

In [ ]:
# for track_id, traj in traj_collection:
#     print("track_id=", track_id)
#     traj.timeframe_to_single_cell[list(traj.timeframe_to_single_cell.keys())[0]].show_panel(figsize=(20, 5))
#     plt.show()
    

In [ ]:
%gui qt
from livecell_tracker.core.napari_visualizer import NapariVisualizer
import napari
from skimage import data


In [57]:
from livecell_tracker.core.single_cell import SingleCellStatic, SingleCellTrajectory, SingleCellTrajectoryCollection
import numpy as np
from napari.viewer import Viewer
from livecell_tracker.core.visualizer import Visualizer

from livecell_tracker.core.single_cell import SingleCellStatic, SingleCellTrajectory, SingleCellTrajectoryCollection
import numpy as np
from napari.viewer import Viewer
from livecell_tracker.core.visualizer import Visualizer


class NapariVisualizer:
    def viz_traj(traj: SingleCellTrajectory, viewer: Viewer, viewer_kwargs=None):
        if viewer_kwargs is None:
            viewer_kwargs = dict()
        shapes = traj.get_scs_napari_shapes()
        shape_layer = viewer.add_shapes(shapes, **viewer_kwargs)
        return shape_layer

    @staticmethod
    def map_colors(values, cmap="viridis"):
        import matplotlib
        import matplotlib.cm as cm

        minima = min(values)
        maxima = max(values)

        norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
        mapper = cm.ScalarMappable(norm=norm, cmap=cmap)
        res_colors = [mapper.to_rgba(v) for v in values]
        return res_colors

    def viz_trajectories(
        trajectories: SingleCellTrajectoryCollection,
        viewer: Viewer,
        bbox=False,
        contour_sample_num=100,
        viewer_kwargs=None,
        text_parameters={
            "string": "track_id: {track_id}\n",
            "size": 12,
            "color": "white",
            "anchor": "upper_left",
            "translation": [-2, 0],
        },
    ):
        if viewer_kwargs is None:
            viewer_kwargs = dict()
        all_shapes = []
        track_ids = []
        all_scs = []
        all_scts = []
        for track_id, traj in trajectories:
            traj_shapes, scs = traj.get_scs_napari_shapes(bbox=bbox, contour_sample_num=contour_sample_num, return_scs=True)
            all_shapes.extend(traj_shapes)
            track_ids.extend([int(track_id)] * len(traj_shapes))
            all_scs.extend(scs)
        print("length of all_shapes", len(all_shapes), "length of track_ids", len(track_ids), "length of all_scs", len(all_scs), "length of all_scts", len(all_scts))
        properties = {"track_id": track_ids, "sc": all_scs}
        shape_layer = viewer.add_shapes(
            all_shapes,
            properties=properties,
            face_color=NapariVisualizer.map_colors(properties["track_id"]),
            face_colormap="viridis",
            shape_type="polygon",
            text=text_parameters,
            name="trajectories",
            **viewer_kwargs
        )
        return shape_layer


viewer = napari.view_image(dic_dataset.to_dask(), name='dic_image', cache=True)
shape_layer = NapariVisualizer.viz_trajectories(traj_collection, viewer, contour_sample_num=20)

length of all_shapes 38 length of track_ids 38 length of all_scs 38 length of all_scts 0


### [TODO] Connect two trajectories

In [58]:
def select_prior_shape(event):
    print("current shape layer shape properties: ", event)
    current_properties = shape_layer.current_properties
    assert len(current_properties["sc"]) == 1 and len(current_properties["track_id"]) == 1
    if len(shape_layer.selected_data) > 1:
        print("Please select only one shape at a time for connecting trajectories")
        return
    if len(shape_layer.selected_data) == 0:
        print("No shape selected, please select a shape to connect trajectories")
        return
    selected_shape_index = list(shape_layer.selected_data)[0]
    cur_sc = current_properties["sc"][0]
    cur_track_id = current_properties["track_id"][0]
    cur_sct = traj_collection[cur_track_id]
    
    print("setting face color of selected shape...")
    face_colors = list(shape_layer.face_color)
    face_colors[selected_shape_index] = (1, 0, 0, 1) 
    shape_layer.face_color = face_colors
    shape = shape_layer.data[selected_shape_index]

    # print(shape_layer.data)
    # time = viewer.dims.current_step[0]
    return cur_sct, cur_sc, selected_shape_index

viewer = napari.view_image(dic_dataset.to_dask(), name='dic_image', cache=True)
shape_layer = NapariVisualizer.viz_trajectories(traj_collection, viewer, contour_sample_num=20)
shape_layer.events.current_properties.connect(select_prior_shape)

<function __main__.select_prior_shape(event)>

current shape layer shape properties:  Event
{'track_id': array([13]), 'sc': array([<livecell_tracker.core.single_cell.SingleCellStatic object at 0x00000267F862FDC0>],
      dtype=object)}
setting face color of selected shape...
current shape layer shape properties:  Event
{'track_id': array([13]), 'sc': array([<livecell_tracker.core.single_cell.SingleCellStatic object at 0x00000267F862FDC0>],
      dtype=object)}
setting face color of selected shape...
current shape layer shape properties:  Event
{'track_id': array([12]), 'sc': array([<livecell_tracker.core.single_cell.SingleCellStatic object at 0x00000267F862FE50>],
      dtype=object)}
setting face color of selected shape...
current shape layer shape properties:  Event
{'track_id': array([12]), 'sc': array([<livecell_tracker.core.single_cell.SingleCellStatic object at 0x00000267F862FE50>],
      dtype=object)}
setting face color of selected shape...
current shape layer shape properties:  Event
{'track_id': array([10]), 'sc': array([

In [55]:
shape = shape_layer.data[0]

In [56]:
shape_layer.face_color = [(0, 0, 0, 1)] * len(shape_layer.data)

In [ ]:
# viewer.layers.selection.events.active.connect(lambda x: print(dir(x)))